In [512]:
import time
import random
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

# 2.1 Produce the list of neighborhood names along with their URL's

In [377]:
url ='http://www.city-data.com/indexes/neighborhoods/FL/1/'
p = requests.get(url)
b = BeautifulSoup(p.content, "html.parser")

In [378]:
Hoods = b.find(class_ = 'index')
Hoods_links=Hoods.find_all('a')
#Hoods_links

In [379]:
links=[]
for link in Hoods_links:
    if link.get('href').__contains__('Tampa')==True:
        link='http://www.city-data.com'+link.get('href')
        links.append(link)
#links

In [380]:
Names=[]
for link in Hoods_links:
    if link.get('href').__contains__('Tampa')==True:
        x=link.get('href')
        x=x[14:-14]
        Names.append(x)
#Names

In [ ]:
def Names_and_urls(url):
    p = requests.get(url)
    b = BeautifulSoup(p.content, "html.parser")
    column_names=['Name','Link']
    Links_Names=[]
    Hoods = b.find(class_ = 'index')
    Hoods_links=Hoods.find_all('a')
    
    for link in Hoods_links:
        if link.get('href').__contains__('Tampa')==True:
            x=link.get('href')
            x=x[14:-14]
            if link.get('href').__contains__('Tampa')==True:
                Link='http://www.city-data.com'+link.get('href')
                d = dict(zip(column_names,[x,Link]))
                Links_Names.append(d)
    return Links_Names

#Names_and_urls('http://www.city-data.com/indexes/neighborhoods/FL/1/')

In [ ]:
url = 'http://www.city-data.com/indexes/neighborhoods/FL/%d/'

l = []
for page in range(1,134):
    print(url%page)
    l = l + Names_and_urls(url%page)
    time.sleep(2)
    

In [ ]:
l

In [16]:
df=pd.DataFrame(l)
df.set_index('Name', inplace=True)
df.to_csv('LinksandNames.csv')
df.shape

(2393, 1)

In [17]:
%cat LinksandNames.csv | head -10

Name,Link
1000-Channelside-Condo,http://www.city-data.com/neighborhood/1000-Channelside-Condo-Tampa-FL.html
1301-Soho-Condo,http://www.city-data.com/neighborhood/1301-Soho-Condo-Tampa-FL.html
1508-South-Howard-Condo,http://www.city-data.com/neighborhood/1508-South-Howard-Condo-Tampa-FL.html
1611-6th-Avenue-Townhomes,http://www.city-data.com/neighborhood/1611-6th-Avenue-Townhomes-Tampa-FL.html
2002-Dekle-Condo,http://www.city-data.com/neighborhood/2002-Dekle-Condo-Tampa-FL.html
215-Verne-Condo,http://www.city-data.com/neighborhood/215-Verne-Condo-Tampa-FL.html
2513-Maryland-Avenue-Condo,http://www.city-data.com/neighborhood/2513-Maryland-Avenue-Condo-Tampa-FL.html
2515-Maryland-Avenue-Condo,http://www.city-data.com/neighborhood/2515-Maryland-Avenue-Condo-Tampa-FL.html
2518-W-Kansas-Avenue-Condo,http://www.city-data.com/neighborhood/2518-W-Kansas-Avenue-Condo-Tampa-FL.html
cat: write error: Broken pipe


# 2.2 Scrape neighborhood data based on URL

In [573]:
url ='http://www.city-data.com/neighborhood/Alice-Kelly-s-Tampa-FL.html'
url = 'http://www.city-data.com/neighborhood/40th--A-Lincoln-NE.html'
p = requests.get(url)
b = BeautifulSoup(p.content, "html.parser")

In [574]:
for item in l:
    x=item
    for item in x.values():
        links2.append(item)
names=links2[0::2]
#names

In [575]:
#### gets the area and population
def area_population(url):
    try:
        p = requests.get(url)
        b = BeautifulSoup(p.content, "html.parser")
        first=b.find(class_='row')
        second=first.find(class_='content-item')
        third=second.text.split('\n')
        fourth=third[0].split(' ')
        fifth=fourth[1:]
        space=' '
        area=space.join(fifth)
    except:
        area='NA'
    try:
        p = requests.get(url)
        b = BeautifulSoup(p.content, "html.parser")
        first=b.find(class_='row')
        second=first.find(class_='content-item')
        third=second.text.split('\n')
        sixth=third[1].split(' ')
        population=sixth[1]
    except:
        population='NA'
    return [area,population]
area_population('http://www.city-data.com/neighborhood/5th-Avenue-Lofts-Condo-Tampa-FL.html')

['1', 'density:5th']

In [576]:
#Finding area to search for data
first=b.find(class_='row')
second=first.find_all(class_='content-item')
third=first.find_all(class_='hgraph')
third[0]

<div class="hgraph"><table><tr><td><b>40th  A:</b></td><td><p class="h" style="padding-left:150px;"></p>5,532 <b>people per square mile</b></td></tr><tr><td><b>Lincoln:</b></td><td><p class="a" style="padding-left:101px;"></p>3,756 <b>people per square mile</b></td></tr></table></div>

In [577]:
#finds specific data I want
third[0].find_all('tr')[0].find_all('td')[1].text

'5,532 people per square mile'

In [578]:
#fuction to get all the fields of data that I want
def parse_neigh(url):
    data=[]
    p = requests.get(url)
    b = BeautifulSoup(p.content, "html.parser")
    first=b.find(class_='row')
    second=first.find_all(class_='content-item')
    third=first.find_all(class_='hgraph')
    for ind in range(0,21):
        #move try except out of the loop, too bad just proceed to the next one.
        try:
            x=third[ind].find_all('tr')[0].find_all('td')[1].text
            data.append(x)
            #append the keys with the data instead of later.
        except:
            #pass
            data.append('NA')
    return data
parse_neigh('http://www.city-data.com/neighborhood/Alice-Kelly-s-Tampa-FL.html')

['1,020 people per square mile',
 '$52,778',
 '$911',
 '21',
 '35.2 years',
 '29.3 years',
 '1.3 people',
 '20.7%',
 '10.7%',
 '56.5%',
 '102.7%',
 '32.0%',
 '34.9%',
 '0.0%',
 '28.0%',
 '55.9%',
 '5.3%',
 '10.7%',
 '1.6',
 '1.2',
 '36.8%']

In [579]:
#Turning my data into a dictionary

def data_acquisition(url,name):
    values = [name]+area_population(url)+parse_neigh(url)
    keys = ['Neighborhood Name',
        'Area',
        'Population',
        'Population density',
        'Median household income',
        'Median rent',
        'Population of males',
        'Median age of males',
        'Median age of females',
        'Average household size',
        'Percentage of family households',
        'Percentage of married-couple families',
        'Percentage of married-couple families with children',
        'Percentage of single mother households',
        'Percentage of never married males Over 15',
        'Percentage of never married females Over 15',
        'Percentage of people that do not speak english',
        'Percentage of people born in the state',
        'Percentage of people born in another state',
        'Percentage of native residents born outside the US',
        'Percentage of foreign born residents',
        'Average number of vehicles per house/condo',
        'Average number of vehicles per apartment',
        'Percentage of units with a mortgage']
    d = dict(zip(keys,values))
    total_data.append(d)

In [580]:
#data_acquisition('http://www.city-data.com/neighborhood/Beach-Park-Annex-Tampa-FL.html','name')

# 2.3 Scraping all the neighborhoods

In [581]:
#clears total_data

#total_data=[]

In [582]:
#Obtains all the links to neighborhoods in Tampa
links2=[]
for item in l:
    x=item
    for item in x.values():
        links2.append(item)
links3=links2[1::2]
#links3

In [ ]:
#goes though all the links and adds the name to them and scrapes the data from every site
c=1032
for link in links3[1032:1051]:
    x=random.randint(10,20)
    data_acquisition(link,names[c])
    time.sleep(x)
    c+=1
    print('done')

In [583]:
#total_data[-1]

In [613]:
df2=pd.DataFrame(total_data)
df2.set_index('Neighborhood Name', inplace=True)
df2.to_csv('DataAcquisition.csv')
df2.shape
#df2

(1032, 23)

In [614]:
%cat DataAcquisition.csv | head -10

Neighborhood Name,Area,Average household size,Average number of vehicles per apartment,Average number of vehicles per house/condo,Median age of females,Median age of males,Median household income,Median rent,Percentage of family households,Percentage of foreign born residents,Percentage of married-couple families,Percentage of married-couple families with children,Percentage of native residents born outside the US,Percentage of never married females Over 15,Percentage of never married males Over 15,Percentage of people born in another state,Percentage of people born in the state,Percentage of people that do not speak english,Percentage of single mother households,Percentage of units with a mortgage,Population,Population density,Population of males
1000-Channelside-Condo,,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,242.5,NA
1301-Soho-Condo,0.003 square miles,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,242.5,NA
1508-South-Howard-Condo,,NA,NA,NA,NA,NA,NA,NA